Перед нами стоит задача предсказания числового значения на основе заданного текста. По заголовку мы должны предсказать его числовой рейтинг от 0.0 до 10.0 с точностью 0.1. 

В качестве библиотеки, реализующей модель русского языка, мы используем spaCy (в первую очередь из соображений скорости).

- Лемматизация
- Составление триграмм
- Категоризация отрезков текста

In [1]:
import pandas as pd

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# https://spacy.io/models/ru
import spacy
nlp = spacy.load('ru_core_news_sm')

In [2]:
doc = nlp('Я умею, летаю')
for token in doc:
    print('{} --> {} --> {}'.format(token, token.lemma_, token.pos_))

Я --> я --> PRON
умею --> уметь --> VERB
, --> , --> PUNCT
летаю --> летать --> VERB


In [4]:
pd.read_csv('../../DATASETS/tproger.csv', index_col=0, parse_dates=['post_time', 'parse_time'])

,title,post_time,short_text,views_num,source,parse_time,filename
https://tproger.ru/digest/7-mobilnyh-prilozhenij-dlja-izuchenija-programmirovanija/,7 мобильных приложений для изучения программир...,2021-03-12 15:23:47+03:00,Собрали для вас актуальные мобильные приложени...,4863,tproger,2021-03-14,a8e7d0e3-07f3-589f-b037-41af38595494
https://tproger.ru/articles/moja-istorija-v-it-kak-samostojatelno-izuchit-javascript-i-python-chtoby-osushhestvit-svoju-mechtu/,Моя история в IT: как самостоятельно изучить J...,2021-03-12 14:08:22+03:00,Я закончил университет по специальности «Специ...,1197,tproger,2021-03-14,e177b763-9c88-5ab3-8879-83100f00ad33
https://tproger.ru/books/podborka-biznes-literatury-dlja-predprinimatelej/,Подборка бизнес-литературы для предпринимателей,2021-03-12 09:07:27+03:00,"Хочется верить, что прочитанные книги положите...",174,tproger,2021-03-14,325e66ce-8f7e-570c-85f6-983db7ba8786
https://tproger.ru/articles/discord-bot-na-python-dlja-avtomatizacii-raboty-s-unity-cloud-build-v-gamedev-komande/,Discord-бот на Python для автоматизации работы...,2021-03-11 18:33:48+03:00,"Я расскажу о создании Discord-бота на Python, ...",972,tproger,2021-03-14,39d92c4a-644f-5fd4-8e2a-613be518437a
https://tproger.ru/articles/11-zadach-po-sql-s-sobesedovanij-na-poziciju-data-engineer/,11 задач по SQL с собеседований на позицию Dat...,2021-03-11 11:16:34+03:00,Позиция data-инженера востребована в IT-индуст...,1518,tproger,2021-03-14,81f6f012-670a-559e-b423-e5e3998403e4
...,...,...,...,...,...,...,...
https://tproger.ru/problems/string-algorithms/,Подборка полезных алгоритмов для собеседований...,2015-02-12 11:48:02+03:00,В этом материале мы решили разобрать алгоритмы...,12108,tproger,2021-03-14,6bf7c355-ab56-5e89-bb53-21ab975cfc95
https://tproger.ru/translations/art-of-structure-packing/,Искусство упаковки структур в C,2015-02-12 11:14:05+03:00,От переводчика Объем памяти и скорость процесс...,42509,tproger,2021-03-14,d14fb48e-e0ed-5719-aa57-2d13e2fa7590
https://tproger.ru/interview/james-bottomley-answers/,Ответы Джеймса Боттомли на вопросы подписчиков...,2015-02-12 02:11:02+03:00,Недавно мы публиковали интервью с Джеймсом Бот...,629,tproger,2021-03-14,3ad8e11e-2dab-54a3-b0f2-b37205011337
https://tproger.ru/problems/how-far-a-fleet-of-n-trucks-can-go/,Как далеко вы сможете доставить груз на N груз...,2015-02-11 12:12:11+03:00,У вас есть парк из 50 грузовиков. Каждый из ни...,30780,tproger,2021-03-14,26cec15c-7437-55d4-9f75-e517bf867ff8
